### Copyright-protected material, all rights reserved. (c) University of Vienna.
_Copyright Notice of the corresponding course at Moodle applies. <br> Only to be used in the MRE course._

# MRE Assignment 1 - Digital Image Processing

In this assignment you will use Python (using Pillow or OpenCV) to load, transcode and store images. You will also use the libraries to extract some basic metadata from the images and store them in a data structure for easy access.

In this notebook, you find the detailed specification. For assessment of your solution you are expected to demonstrate your implementation and answer questions in mostly textual form here.

❗ **Note:** Please make sure that all potential errors, including handling files, paths, and run-time errors are handled properly (e.g., useful error messages to users).

## Import your implementation

Import the corresponding Jupyter Notebook named "*_impl.ipynb" for this assignment here.

In [18]:
%%capture
%run MRE_A1_impl.ipynb

## Task 1.1: Image formats transformation and adding filters- 20 pts.

a. Write a Python function `JPEGImageConverter` that accepts 3 parameters: an input file or directory, and output directory name, and an optional quality parameter. If the optional quality parameter is not provided, it defaults to 100% (1.0), i.e., full quality. 

The function should be able to: 
* Read the input image(s).
* Convert the input image(s) to the JPEG file format.
* Store the converted image(s) to the output directory.
  * For BMP and PNG files it is ok to use the default image writer provided by the Python framework.
  * For JPEG a configurable writer has to be used which requires the quality parameter.
  
The output filename should conform to the following structure: <br>
  `<original name>-<quality>.jpeg`
<br>
  
**Examples:** <br>
input: `image1.gif` <br>
output: `image1.gif.jpg` <br>
output: `image1.gif-0.5.jpg`<br>
Function call: `JPEGImageConverter("image1.jpg", "./output_dir", 0.75)`



* Test your function with the files in the media directory.
* Test your function with your own images.
* Use your function to convert the provided test images (please **choose 4 images** for comparison):
  * Convert them to the following formats: JPEG (quality: 0), JPEG (quality: 0.5), JPEG (quality: 1.0).
  * Compare the file sizes of the original and the created files.
  * Compare the quality of the resulting images manually (use "GOOD", "MEDIUM", "BAD" for your subjective assessment).
  
b. Write a Python function `BlurImage` that accepts 1 parameter; an input file name. The function should add a blur filter to the input image and store the resulting blurred image in an output directory.

**Examples:** <br>
input: `image1.gif` <br>
Function call: `BlurImage("image1.jpg")`


### Demonstrate your implementation:

In [19]:
# Demonstrate your implementation here. 
# Only enter the calls to your functions here so you can demonstrate validity of your solution.

JPEGImageConverter("images/b17barb026.jpg", "./output_dir", 0.0)
JPEGImageConverter("images/b17barb026.jpg", "./output_dir", 0.5)
JPEGImageConverter("images/b17barb026.jpg", "./output_dir", 1.0)

JPEGImageConverter("images/ml-1.jpg", "./output_dir", 0.0)
JPEGImageConverter("images/ml-1.jpg", "./output_dir", 0.5)
JPEGImageConverter("images/ml-1.jpg", "./output_dir", 1.0)

JPEGImageConverter("images/m-2.jpg", "./output_dir", 0.0)
JPEGImageConverter("images/m-2.jpg", "./output_dir", 0.5)
JPEGImageConverter("images/m-2.jpg", "./output_dir", 1.0)

JPEGImageConverter("images/zebra3m.JPG", "./output_dir", 0.0)
JPEGImageConverter("images/zebra3m.JPG", "./output_dir", 0.5)
JPEGImageConverter("images/zebra3m.JPG", "./output_dir", 1.0)


BlurImage("images/b17barb026.jpg")


### Additionally, answer the following questions in written form:

**1. Did the file sizes increase or decrease when you used different formats and why?**

When the quality is at 1.0, there is (almost) no loss of quality. With the quality of 0.5, the loss is easily visible. With 0.0, the picture is a complete different one. As JPEG uses Discrete Cosine Transform (DCT), it can reduce certain higher frequencies used for each block and therefore save space. The lower the quality, the more intense the quantisation of thew coefficients. More exact explanation in 3.

**2. What is the difference between lossy and lossless image formats?**

Lossless image formats such as .BMP and .PNG only use lossless compression such as Huffman coding, arithmetic encoding, or run-length encoding. Mostly, the RGB values for every pixel are saved and restored, therefore reconstructing the original image.

In lossy formats, information in the image is lost irreversibly. Altough they use genious techniques to save space and the differences aren't that visible at high quality values, further processing and usage of the image becomes harder.

**3. Explain the individual processing steps of the JPEG method following a complete process scheme as introduced in the course.**

1. The color space is changed to YCbCr, as luminance is more decisive for the human eye than the chrominance values.
2. The chrominance values get downsampled (4 Pixel-blocks), as they make the least difference. (4:2:2 in Video)
3. Pixels get divided into 8x8 blocks.
4. Forward DCT is applied on the blocks. Each pixel in the block now stores certain frequencies in vertical or horizontal directions.
5. Now, the most important step can be applied: quantization.
    Here, the pixels (or entries in the matrix) with higher frequencies can be just set to zero, as they make the least difference for the human eye. The lower the quality value is set, the lower the threshhold for quantizing pixels.
6. Entropy encoding: As there are a lot of "0-pixels" now in the matrix espacially in the high-frequency department, we use Run-Length-Encoding and Huffman Coding to save that space (in a zig-zag manner).

**4. Considering your implementation of the `BlurImage` function, which paramaters can you add to extend the signature of your function?**

A repeat parameter could be added, which would tell the function to repeat the process a certain number of times.


## Task 1.2:  Extract / Get Image Metadata - 20 pts.

Write a Python function `ImageMetadataGenerator` which can store the following extracted metadata in a Python pandas DataFrame. The function can be called with a single parameter: either the name of an input image or the name of a directory in which various input images to be considered can be found. You can use Pillow or OpenCV to complete this task.

The function can store the following metadata: 
- width (integer)
- height (integer)
- number of image components (integer)
- number of channel (integer)
- bits per pixel (integer)
- colorSpaceType (integer)
- additional tags of your choice (max 4 tags).

**Please note:** Sometimes you have to combine other image features to calculate a certain meta-information, for example: extracting the color space or the number of channels might help to calculate bitsPerPixel or image components. bitsPerPixel is the number of bits used to represent the color information for a pixel.

**Example:** <br>
Function call:  `ImageMetadataGenerator("image1.jpg")`

### Demonstrate your Implementation:

In [20]:
# Demonstrate your implementation here.
# Only enter the calls to your functions here so you can demonstrate validity of your solution.
display(ImageMetadataGenerator("images/"))

images/b17barb026_dithered.gif
images/ml-3.jpg
images/ml-1.jpg
images/zebra3m.JPG
images/onb-2.jpg
images/b17maartent1427.gif
images/picture_of_coati_dithered.png
images/onb-3.jpg
images/ml-5.jpg
images/b2seaanimals005.jpg
images/onb-1.jpg
images/jawa-2.2.jpg
images/peafowl_picture1.JPG
images/b3_seaanimals014.jpg
images/par-1.jpg
images/jawa-2.1.jpg
images/ml_2.jpg
images/blue_sky4m.JPG
images/blue_sky5m.JPG
images/vie-1.jpg
images/b20nature_plants245.jpg
images/vie-3.jpg
images/vie-2.jpg
images/b1murene.jpg
images/the_real_firefox.png
images/ca-1.jpg
images/m-1.jpg
images/m-3.jpg
images/picture_of_coati.JPG
images/underwater_rugby.JPG
images/m-2.jpg
images/b9nature_plants064.jpg
images/b17maartent1427.jpg
images/lib-1.jpg
images/the_real_firefox_dithered.bmp
images/b1shell008.jpg
images/ca-21.jpg
images/lib-2.jpg
images/picture_of_goat11.JPG
images/m-4.jpg
images/jawa-4.jpg
images/elephant14m.JPG
images/m-4.1.jpg
images/elephant14m.png
images/lib1-xMedUni_Wien_Josephinum_Bibliothek-m

/var/folders/kv/rlm1b_915wsbvd995jlr3f880000gn/T/ipykernel_941/887948427.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
/var/folders/kv/rlm1b_915wsbvd995jlr3f880000gn/T/ipykernel_941/887948427.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
/var/folders/kv/rlm1b_915wsbvd995jlr3f880000gn/T/ipykernel_941/887948427.py:43: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = df.append({
/var/folders/kv/rlm1b_915wsbvd995jlr3f880000gn/T/ipykernel_941/887948427.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
/var/folders/kv/rlm1b_915wsbvd99

,File,Width,Height,Components,Channels,BitsPerPixel,ColorSpaceType,Format,Pixels,HasAlphaChannel,SizeWithoutCompressionInKB
0,images/b17barb026_dithered.gif,1024,768,1,1,8,P,GIF,786432,False,768.000000
1,images/ml-3.jpg,600,400,3,3,24,RGB,JPEG,240000,False,703.125000
2,images/ml-1.jpg,620,342,3,3,24,RGB,JPEG,212040,False,621.210938
3,images/zebra3m.JPG,800,532,3,3,24,RGB,JPEG,425600,False,1246.875000
4,images/onb-2.jpg,348,348,3,3,24,RGB,JPEG,121104,False,354.796875
5,images/b17maartent1427.gif,50,38,1,1,8,P,GIF,1900,False,1.855469
6,images/picture_of_coati_dithered.png,1023,558,1,1,8,P,PNG,570834,False,557.455078
7,images/onb-3.jpg,480,480,3,3,24,RGB,JPEG,230400,False,675.000000
8,images/ml-5.jpg,600,450,3,3,24,RGB,MPO,270000,False,791.015625
9,images/b2seaanimals005.jpg,1600,1200,3,3,24,RGB,JPEG,1920000,False,5625.000000


## Task 1.3: Drawing circles arund objects in an image - 20 pts

a. Write a Python function `IdentifyObjects` to be called with 1 parameter; an input file. It identifies the objects or shapes in the image and draws a circle around the identified shapes.

**Example:** <br>
Input: `partyballoons-1.jpg` <br>
Function Call: `IdentifyObjects("partyballoons-1.jpg")`


b. Test your function using the image `partyballoons-2.jpg` and explain the outcome.

💡 **TIP**: Use the provided images in the directory "Task1.3" to demonstrate your solution.

### Demonstrate your Implementation:

In [21]:
# Demonstrate your implementation here.
# Only enter the calls to your functions here so you can demonstrate validity of your solution.

#a
IdentifyObjects("images/Task1.3/partyballoons-1.jpg")

#b
IdentifyObjects("images/Task1.3/partyballoons-2.jpg")

**b. Explenation**

First, as colors don't matter here, the color is turnt into grey.
Then, the picture is slightly blurred, to allow for a more precise edge detection.
After that, the edges are identified.
We can now run the function findContours, which gives us a list of contours the program believes to be right.
Having identified the position and size of the contour, we can now draw an enclosing circle around each contour.

In our case, the program detected the outer edges of the balloons as contours.
In the lower two, it also detected the threads of the balloons as a part of the contours, in the upper three, it detected just the balloons itself.

## Task 1.4: Duplicate image finder using cosine similarity - 40 pts.

a. Write a Python function `ImageHistogramGenerator` to be called with 1 parameter; an input file. It returns the object that represents the calculated histogram.

**Example:** <br>
Input: `image1.jpg` <br>
Function Call: `ImageHistogramGenerator("image1.jpg")`

💡 **TIP**: You may plot your histogram using `matplotlib` to check if it looks correct. However, please be sure to comment out / disable your plots when calling the following `FindDuplicateImages` function, since they may cause memory issues on the JupyterLab.


b. Write a Python function `FindDuplicateImages` that can be called with three parameters:
- `inputDir1`: the path to the first directory, which contains all the images.
- `inputDir2`: the path to the second directory, which contains a subset of the images.
- `similarityThreshold`: a threshold parameter, which ranges between 0.0 to 1.0.

The algorithm:
- Generate the histograms for all images in the input directory 1 and input directory 2.
- Calculate the cosine distance between the histogram of the images in the input directory 2 and all other images in the input directory 1.

- Generate a pandas DataFrame and display it for visualization of the comparison results. The DataFrame should consist of the following three columns:

  - `inputDir1`: displays the image name in the directory.
  - `inputDir2`: displays the image name in the directory.
  - `similarityThreshold`: displays the similarity (0.0 - 1.0)

**Example:**<br>
Function call: `FindDuplicateImages("inputDir1", "inputDir2","similarityThreshold")` <br>


💡 **TIP**: To keep the memory footprint and runtime of your solution low, consider limiting the amount of files in the `inputDir2`.

### Additionally, answer the following questions in written form:

**1. Can the created histogram be used to reconstruct the image? If no, what other information would be needed?**

Your answer here.

**2. Explain the differences between an individual color histogram, a combined color histogram and an intensity histogram.** <br>
&emsp; **a) What information is displayed in each of these diagrams?** <br>
&emsp; **b) What histogram type did we use?** <br>

Your answer here. 

**3. Explain the concept of histogram bins.**

Your answer here.

### Demonstrate your Implementation:

In [22]:
# Demonstrate your implementation here. 
# Only enter the calls to your functions here so you can demonstrate validity of your solution.
